## Book to download and store schedule table from college hockey news


### Dependencies and Setup
- 

In [9]:
## This workbook is used to scrape the data from the College Hockey News and explore the data## 

######## DONT SEEM TO BE ENFORCED ########
## Notes - the site's robots.txt file sets these limits for crawlers
# Crawl-delay: 10 (seconds)
# Request-rate: 1/5 (5 requests every minute)

# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

# Set the URL to scrape
url_base = 'https://www.collegehockeynews.com/schedules/?season=' # Link to the 2022-2023 season with all results

#### THESE NEED TO BE UPDATED EACH SEASON ####
#### Set the season varriable portion of the url 
url_season = '20232024'
#### string to add to file names and whatnote
file_tage = 'Current Season 2023'

############# Set the output folder
temp_output_dir = '../TEMP/results/'
final_output_dir = '../data/schedule_results/'

## Construction teh url to feel to parser
url = url_base + url_season


## Request the page and parse the html for the Schedule Table

- table structure requires that we extract information about things like the data for the contest and Conference to note has to be extracted from header rows withing the larger table.
    - Overcome this by looking at the html row by row and store the data from the headers to apply to the rows found below it

In [10]:
# Get the page with requests
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# select the table or tables
tables = soup.find_all('table')

# Initialize variables
current_date = None
current_conference = None
game_notes = None

# Initialize an empty list to hold the data
data = []

# Parse the table with BeautifulSoup

rows = soup.find_all('tr')

# Loop through each row to find relevant information
for row in rows:
    # Check for date row
    if row.get('class') == ['stats-section']:
        current_date = row.find('td').text.strip()
    # Check for conference row
    elif row.get('class') == ['sked-header']:
        current_conference = row.find('td').text.strip()
    # Check for game notes
    elif len(row.find_all('td')) == 2:
        game_notes = row.find_all('td')[1].text.strip()
    # Process rows with game data
    elif row.get('valign') == 'top':
        cells = row.find_all('td')
        if len(cells) >= 9:
            home_team = cells[0].text.strip()
            home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
            home_score = cells[1].text.strip()
            away_team = cells[3].text.strip()
            away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
            away_score = cells[4].text.strip()
            ot = cells[5].text.strip()
            box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
            metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
             # Capture Game Notes
            game_notes_cell = cells[-1].find('small')
            game_notes = game_notes_cell.text.strip() if game_notes_cell else None

            # Append data to the list
            data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
            game_notes = None  # Reset game notes for the next row
            

# Create a DataFrame
columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)



## Do some light data transformation

In [11]:
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)


C:\Users\Justin\AppData\Local\Temp\ipykernel_18456\2220396521.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-10-06_Arizona_State' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row.Index, 'Game_ID'] = game_id


In [12]:
# ### Look at Output

# df.head()

# df.sample(10)

# df.info()

### Save the Dataframe collected to a CSV file 
- use the dir variable to put it in temp file or final data folder

In [13]:
### OUTPUT A TEMP FILE TO THE TEMP FOLDER
temp_file_name = f'{temp_output_dir}temp_schedule_results_{file_tage}.csv'

### OUTPUT A FINAL FILE TO THE DATA FOLDER AFTER MANUAL CHECKING
final_file_name = f'{final_output_dir}schedule_results_{file_tage}.csv'

## Load the Roster File to do some work

- Step 1 Load
- Step 2 - take the DOB column and convert it into age in days and age in years to make calculations easier

In [22]:
roster_path = '../data/rosters/2023_master_roster.csv'

# Read in the roster file
roster_df = pd.read_csv(roster_path)

## Convert and store age as number of days old
roster_df['Age'] = pd.to_datetime(roster_df['DOB']).dt.strftime('%Y-%m-%d')
roster_df['Age_Days'] = pd.to_datetime(roster_df['Age'])

## Calculate age in days
roster_df['Age_Days'] = pd.to_datetime('today') - roster_df['Age_Days']

# Simplify the Age_Days to just the number of days
roster_df['Age_Days'] = roster_df['Age_Days'].dt.days

# Create Age_Years column with float values
roster_df['Age_Years'] = roster_df['Age_Days'] / 365.25

# LOOK AT DATA
roster_df.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School,Age,Age_Days,Age_Years
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American-Intl,2003-06-23,7428,20.336756
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American-Intl,1998-10-16,9139,25.021218
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American-Intl,2000-07-20,8496,23.260780
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American-Intl,2003-04-07,7505,20.547570
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American-Intl,2001-08-09,8111,22.206708


## Group by Team and claculate the average age in Years

In [23]:
## Group by team and calculate TEAM_AVG_AGE
team_age_df = roster_df.groupby('Team', as_index=False).agg({'Age_Years': 'mean'})
# Sort the DataFrame by Age_Years
team_age_df = team_age_df.sort_values('Age_Years', ascending=False)

# Group by Team and Position and calculate TEAM_POS_AVG_AGE
team_pos_age_df = roster_df.groupby(['Team', 'Position'], as_index=False).agg({'Age_Years': 'mean'})
# Sort the DataFrame by Age_Years
team_pos_age_df = team_pos_age_df.sort_values('Age_Years', ascending=False)

### Map the numerical values for Yr. column
## Create a dictionary to map the values
yr_map = {
    'Fr': 1,
    'So': 2,
    'Jr': 3,
    'Sr': 4,
    'Gr': 5
}

# Map the values to the DataFrame
roster_df['Yr'] = roster_df['Yr'].map(yr_map)

# Group by Team and calculate TEAM_AVG_YR
team_yr_df = roster_df.groupby('Team', as_index=False).agg({'Yr': 'mean'})
# Sort the DataFrame by Yr
team_yr_df = team_yr_df.sort_values('Yr', ascending=False)

# Group by Team and Position and calculate TEAM_POS_AVG_YR
team_pos_yr_df = roster_df.groupby(['Team', 'Position'], as_index=False).agg({'Yr': 'mean'})
# Sort the DataFrame by Yr
team_pos_yr_df = team_pos_yr_df.sort_values('Yr', ascending=False)

# LOOK AT DATA
### Age In Years
team_age_df.head()
team_pos_age_df.head()

### Experience in Years/Seasons
team_yr_df.head()
team_pos_yr_df.head()


### Merge all the dataframes together
## Merge the team_age_df and team_yr_df

team_age_experience_averages_df = pd.merge(team_age_df, team_yr_df, on='Team', how='outer')
# Add the next
team_age_experience_averages_df = pd.merge(team_age_experience_averages_df, team_pos_age_df, on=['Team', 'Position'], how='outer')
# Add the next
team_age_experience_averages_df = pd.merge(team_age_experience_averages_df, team_pos_yr_df, on=['Team', 'Position'], how='outer')


# LOOK AT DATA
team_age_experience_averages_df.head()



KeyError: 'Team'